In [1]:
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from glob import glob
from timeit import default_timer as timer
import sys
import scipy.stats as stats

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from tune_sklearn import TuneGridSearchCV

from keras.utils import np_utils,to_categorical
from keras.optimizers import RMSprop, Adam
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import optimizers

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

In [2]:
import sys
import os
import csv
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox, Button, RadioButtons
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec

from sklearn import preprocessing
from sklearn import decomposition

In [3]:
class rawDataset():
    def __init__(self, device, movename, timestamp, dataset):
        self.device = device
        self.movename = movename
        self.timestamp = timestamp
        self.dataset = dataset
    def plot(self):
        fig = plt.figure()
        gs = gridspec.GridSpec(3, 1, width_ratios=[1], height_ratios=[0.2,1,1])
        index = [ x for x in range(len(self.dataset['activation_List']))]
        # print(index)
        # TODO plot the activation
        activation = fig.add_subplot(gs[0])
        accel = fig.add_subplot(gs[1])
        gyro = fig.add_subplot(gs[2])

        activation.set_title("Device " + str(self.device) + "\nActivation" )
        accel.set_title("Accel" )
        gyro.set_title("Gyro" )


        ax1, = accel.plot(index, self.dataset['a_xList'], label = "X")
        ax2, = accel.plot(index, self.dataset['a_yList'], label = "Y")
        ax3, = accel.plot(index, self.dataset['a_zList'], label = "Z")

        ax4, = gyro.plot(index, self.dataset['g_xList'], label = "X")
        ax5, = gyro.plot(index, self.dataset['g_yList'], label = "Y")
        ax6, = gyro.plot(index, self.dataset['g_zList'], label = "Z")

        ax7, = activation.plot(index, self.dataset['activation_List'], label = "R")
        
        displaylen = len(self.dataset['activation_List'])

        activation.set_xlim(xmin = 0 , xmax = displaylen )
        activation.set_ylim(ymin = 0.2 , ymax = 2.2 )

        accel.set_xlim(xmin = 0 , xmax = displaylen )
        accel.set_ylim(ymin = -2 , ymax = 2 )

        gyro.set_xlim(xmin = 0 , xmax = displaylen )
        gyro.set_ylim(ymin = -250 , ymax = 250 )
        
        # plt.show()
        # plt.clf()

In [5]:
pd.set_option("display.max_rows", 100, "display.max_columns", 100)
np.set_printoptions(threshold=100) # 'sys.maxsize' for max

In [6]:
train_dancers = ['Abi', 'CJ', 'Ryan', 'Alex', 'XY', 'Sean']
test_dancers = ['Alex', 'XY', 'Sean']

In [7]:
def dancer_data_train(dancers):
    df = pd.DataFrame()
    for dancer in dancers:
        print("\nProcessing {}'s dance moves as training set:" .format(dancer))
        df_temp = processData(dancer, testset=False)
        df_temp['Dancer'] = dancer
        df = pd.concat([df, df_temp], axis=0, ignore_index=True)
    return df

def dancer_data_test(dancers):
    df = pd.DataFrame()
    for dancer in dancers:
        print("\nProcessing {}'s dance moves as test set:" .format(dancer))
        df_temp = processData(dancer, testset=True)
        df_temp['Dancer'] = dancer
        df = pd.concat([df, df_temp], axis=0, ignore_index=True)
    return df

In [8]:
dance_move_training = dancer_data_train(train_dancers)


Processing Abi's dance moves as training set:
Recording from dev2 with move pointhigh at 16177872911030 opened with 490 samples
Device dev2recal connected!
dev2recal activation | G:0.6677268534359555 | zG:-0.25512625570046743
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev3 with move elbowkick at 16176152112068 opened with 489 samples
Device dev3recal connected!
dev3recal activation | G:0.40745843714805585 | zG:-0.1862640009914891
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev2 with move elbowkick at 16177865938445 opened with 485 samples
Device dev2recal connected!
dev2recal activation | G:0.479885009849248 | zG:-0.09662003383905926
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move hair at 16177868198639 opened with 490 samples
Device dev2recal connected!
dev2recal activation | G:0.540443773521482 | zG:-0.16593386542330735
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move wipetable at 16177877081388 opened with 52

Recording from dev2 with move elbowkick at 16177864640337 opened with 574 samples
Device dev2recal connected!
dev2recal activation | G:0.5904155498305976 | zG:0.12417856506183034
dev2recal MoveTimout
dev2recal Too Short!
dev2recal activation | G:0.4233661408806473 | zG:0.09030501432995952
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev3 with move pointhigh at 16176164437663 opened with 444 samples
Device dev3recal connected!
dev3recal activation | G:0.41279644070713406 | zG:-0.2589779880756389
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev3 with move elbowkick at 16176151536222 opened with 495 samples
Device dev3recal connected!
dev3recal activation | G:0.4409615676507217 | zG:-0.15317455452291284
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev2 with move sidepump at 16177874662800 opened with 561 samples
Device dev2recal connected!
dev2recal activation | G:0.5032533367146632 | zG:-0.4384137202942442
dev2recal MoveTimout
dev2recal Too Short!
Reco

478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
548
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
706
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
494
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
569
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
593
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670
670


Recording from dev2 with move wipetable at 16176170420368 opened with 481 samples
Device dev2recal connected!
dev2recal activation | G:0.43290162465742266 | zG:-0.03536744882964249
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move pointhigh at 16176165305703 opened with 447 samples
Device dev2recal connected!
dev2recal activation | G:0.45988641751943127 | zG:-0.22579599693130525
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev1 with move dab at 16177863171526 opened with 510 samples
Device dev1recal connected!
dev1recal activation | G:0.5305339115302787 | zG:-0.12011987406948621
dev1recal MoveTimout
dev1recal Too Short!
Recording from dev2 with move listen at 16176161924496 opened with 671 samples
Device dev2recal connected!
dev2recal activation | G:0.5512903953311508 | zG:-0.1454751272672823
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev1 with move pointhigh at 16177873215488 opened with 472 samples
Device dev1recal connected!
dev1recal 

562
562
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
496
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
552
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
488
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
478
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
580
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
532
442
442
442
442
442
442
442
442
442
442
442
442
442
442
442
442
442


dev3recal MoveTimout
dev3recal Too Short!
Recording from dev1 with move listen at 16176162651081 opened with 516 samples
Device dev1recal connected!
dev1recal activation | G:0.6601856024702168 | zG:-0.22653076525975901
dev1recal MoveTimout
dev1recal Too Short!
Recording from dev1 with move dab at 16176146629308 opened with 463 samples
Device dev1recal connected!
dev1recal activation | G:0.7740190898568107 | zG:-0.3795502292092494
dev1recal MoveTimout
dev1recal Too Short!
Recording from dev1 with move gun at 16176158289494 opened with 503 samples
Device dev1recal connected!
dev1recal activation | G:0.5257608993536438 | zG:-0.060007176229997405
dev1recal MoveTimout
dev1recal Too Short!
Recording from dev1 with move listen at 16176161924397 opened with 674 samples
Device dev1recal connected!
dev1recal activation | G:0.5535272014056707 | zG:-0.1226863869350634
dev1recal MoveTimout
dev1recal Too Short!
Recording from dev1 with move sidepump at 16176166959198 opened with 529 samples
Device d

613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
613
???
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
534
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
529
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
497
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
443
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
519
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
574
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508
508


444
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
468
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
486
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
422
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
515
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
520
411
411
411
411
411
411
411
411
411
411
411
411
411
411
411
411
411
411


dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move dab at 16177849969674 opened with 544 samples
Device dev2recal connected!
dev2recal activation | G:0.43854845243523055 | zG:-0.10535627492508737
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move pointhigh at 16177856321513 opened with 396 samples
Device dev2recal connected!
dev2recal activation | G:0.43962782403747536 | zG:-0.09256376536859445
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move hair at 16177853568776 opened with 452 samples
Device dev2recal connected!
dev2recal activation | G:0.7317790180353305 | zG:-0.4967387599854882
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move sidepump at 16177858158722 opened with 405 samples
Device dev2recal connected!
dev2recal activation | G:0.556676915071155 | zG:-0.3739050381406339
dev2recal MoveTimout
dev2recal Too Short!
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493
493

Recording from dev3 with move gun at 16177852956415 opened with 418 samples
Device dev3recal connected!
dev3recal activation | G:0.5574753346225962 | zG:-0.3026249999999999
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev3 with move gun at 16177852238228 opened with 446 samples
Device dev3recal connected!
dev3recal activation | G:0.7661397466359061 | zG:-0.4042451016239337
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev3 with move elbowkick at 16177850295130 opened with 404 samples
Device dev3recal connected!
dev3recal activation | G:0.5165166890387836 | zG:-0.08601413710594152
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev3 with move listen at 16177855753472 opened with 415 samples
Device dev3recal connected!
dev3recal activation | G:0.46119517450486347 | zG:-0.11370341031550613
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev3 with move hair at 16177853828762 opened with 500 samples
Device dev3recal connected!
dev3recal activation | 

In [ ]:
dance_move_training

In [9]:
dance_move_testing = dancer_data_test(test_dancers)


Processing Alex's dance moves as test set:
Recording from dev1 with move pointhigh at 16176184173502 opened with 356 samples
Device dev1recal connected!
dev1recal activation | G:-0.5612035310590853 | zG:0.5983871065478004
dev1recal MoveTimout
dev1recal Too Short!
dev1recal activation | G:0.4423926943783145 | zG:0.1549344850102382
dev1recal MoveTimout
dev1recal Too Short!
Recording from dev1 with move pointhigh at 16176183962263 opened with 374 samples
Device dev1recal connected!
dev1recal activation | G:0.5470704529846626 | zG:-0.025176636784428386
dev1recal MoveTimout
dev1recal Too Short!
Recording from dev1 with move elbowkick at 16176180261757 opened with 449 samples
Device dev1recal connected!
dev1recal activation | G:0.6871823452693637 | zG:0.01893962483290912
dev1recal MoveTimout
dev1recal Too Short!
Recording from dev1 with move sidepump at 16176184509627 opened with 381 samples
Device dev1recal connected!
dev1recal activation | G:0.4253463045497059 | zG:-0.042357461274961095
d


Processing XY's dance moves as test set:
Recording from dev3 with move hair at 16176182010233 opened with 346 samples
Device dev3recal connected!
dev3recal activation | G:0.9218447233703246 | zG:-0.5267284240000183
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev3 with move pointhigh at 16176183962352 opened with 406 samples
Device dev3recal connected!
dev3recal activation | G:0.46329573787508993 | zG:-0.27032678096773033
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev3 with move listen at 16176182951004 opened with 394 samples
Device dev3recal connected!
dev3recal activation | G:0.4129259768308784 | zG:-0.05787500003867896
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev3 with move listen at 16176183386231 opened with 384 samples
Device dev3recal connected!
dev3recal activation | G:0.7009992823335971 | zG:-0.16116466099765114
dev3recal MoveTimout
dev3recal Too Short!
Recording from dev3 with move sidepump at 16176184722438 opened with 406 samples



Processing Sean's dance moves as test set:
Recording from dev2 with move pointhigh at 16176183743760 opened with 389 samples
Device dev2recal connected!
dev2recal activation | G:0.4473682093649205 | zG:-0.21827296706339783
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move dab at 16176179457991 opened with 461 samples
Device dev2recal connected!
dev2recal activation | G:0.4981333371034662 | zG:-0.28176761403297124
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move dab at 16176179158461 opened with 392 samples
Device dev2recal connected!
dev2recal activation | G:0.5358175041508246 | zG:-0.38096186925027076
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move wipetable at 16176186098283 opened with 408 samples
Device dev2recal connected!
dev2recal activation | G:0.5330573789840347 | zG:-0.4395473703831321
dev2recal MoveTimout
dev2recal Too Short!
Recording from dev2 with move gun at 16176181424055 opened with 423 samples
Devic

446
446
446
446
446
446
446
446
446
446
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
436
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
495
Done
Extracted 912 moves from 24 raw data Sequences.
114  dab
114  elbowkick
114  gun
114  hair
114  listen
114  pointhigh
114  sidepump
114  wipetable
0  defaultMove


In [ ]:
dance_move_testing

In [10]:
import statistics
from spectrum import aryule
from scipy.stats import pearsonr
from scipy.stats import entropy
from math import sqrt
from scipy.fft import fft, fftfreq
from scipy.stats import kurtosis
from scipy.stats import skew

In [11]:
features_train = pd.DataFrame()
features_test = pd.DataFrame()

In [12]:
def get_mean(x):
    data = []
    data_temp = []
    for col in range(x.shape[1]):
        for row in range(x.shape[0]):
            data_temp.append(np.mean(x.iloc[row, col]))
            if row == x.shape[0]-1:
                data.append(data_temp)
                data_temp = []
    return np.array(data).T

def get_std(x):
    data = []
    data_temp = []
    for col in range(x.shape[1]):
        for row in range(x.shape[0]):
            data_temp.append(np.std(x.iloc[row, col]))
            if row == x.shape[0]-1:
                data.append(data_temp)
                data_temp = []
    return np.array(data).T

def get_ptp(x):
    data = []
    data_temp = []
    for col in range(x.shape[1]):
        for row in range(x.shape[0]):
            data_temp.append(np.ptp(x.iloc[row, col]))
            if row == x.shape[0]-1:
                data.append(data_temp)
                data_temp = []
    return np.array(data).T

def fourier_transform(x):
    data = []
    data_temp = []
    for col in range(x.shape[1]):
        for row in range(x.shape[0]):
            data_temp.append(np.fft.fft(xyz_acc_mean.values[row][col]))
            if row == x.shape[0]-1:
                data.append(data_temp)
                data_temp = []
    return np.array(data).T

def get_mag(val):
    data = []
    for i in range(len(val)):
        x, y, z = val.values[i]
        data.append(sqrt(pow(x, 2) + pow(y, 2) + pow(z, 2)))
    return np.array(data).T

def get_kurtosis(x):
    data = []
    data_temp = []
    for col in range(x.shape[1]):
        for row in range(x.shape[0]):
            data_temp.append(kurtosis(x.iloc[row, col]))
            if row == x.shape[0]-1:
                data.append(data_temp)
                data_temp = []
    return np.array(data).T

def get_skew(x):
    data = []
    data_temp = []
    for col in range(x.shape[1]):
        for row in range(x.shape[0]):
            data_temp.append(skew(x.iloc[row, col]))
            if row == x.shape[0]-1:
                data.append(data_temp)
                data_temp = []
    return np.array(data).T 

In [14]:
def extract_features(dataset):
    xyz_acc = dataset.iloc[0:, 0:3]
    xyz_gyro = dataset.iloc[0:, 4:7]
    dancer = dataset.iloc[0:, 8]
    move = dataset.iloc[0:, 7]
    xyz_acc_mean = pd.DataFrame(get_mean(xyz_acc), columns=['acc_X_mean', 'acc_Y_mean', 'acc_Z_mean'])
    xyz_acc_std = pd.DataFrame(get_std(xyz_acc), columns=['acc_X_std', 'acc_Y_std', 'acc_Z_std'])
    xyz_acc_mag = pd.DataFrame(get_mag(xyz_acc_mean), columns=['acc_mag'])
    xyz_acc_ptp = pd.DataFrame(get_ptp(xyz_acc), columns=['acc_X_ptp', 'acc_Y_ptp', 'acc_Z_ptp'])
    
    xyz_acc_kurtosis = pd.DataFrame(get_kurtosis(xyz_acc), columns=['acc_X_kurtosis', 'acc_Y_kurtosis', 'acc_Z_kurtosis'])
    xyz_acc_skew = pd.DataFrame(get_skew(xyz_acc), columns=['acc_X_skew', 'acc_Y_skew', 'acc_Z_skew'])
    
    xyz_gyro_mean = pd.DataFrame(get_mean(xyz_gyro), columns=['gyro_X_mean', 'gyro_Y_mean', 'gyro_Z_mean'])
    xyz_gyro_std = pd.DataFrame(get_std(xyz_gyro), columns=['gyro_X_std', 'gyro_Y_std', 'gyro_Z_std'])
    xyz_gyro_mag = pd.DataFrame(get_mag(xyz_gyro_mean), columns=['gyro_mag'])
    xyz_gyro_ptp = pd.DataFrame(get_ptp(xyz_gyro), columns=['gyro_X_ptp', 'gyro_Y_ptp', 'gyro_Z_ptp'])
    
    xyz_gyro_kurtosis = pd.DataFrame(get_kurtosis(xyz_gyro), columns=['gyro_X_kurtosis', 'gyro_Y_kurtosis', 'gyro_Z_kurtosis'])
    xyz_gyro_skew = pd.DataFrame(get_skew(xyz_gyro), columns=['gyro_X_skew', 'gyro_Y_skew', 'gyro_Z_skew'])
    
#     extracted_features = pd.concat([xyz_acc_mean, xyz_acc_std, xyz_acc_mag,
#                                     xyz_gyro_mean, xyz_gyro_std, xyz_gyro_mag,
#                                     dancer, move], axis=1)
    
    
    extracted_features = pd.concat([xyz_acc_mean, xyz_acc_std, xyz_acc_mag, xyz_acc_ptp,
                                xyz_acc_kurtosis, xyz_acc_skew,
                                xyz_gyro_mean, xyz_gyro_std, xyz_gyro_mag, xyz_gyro_ptp,
                                xyz_gyro_kurtosis, xyz_gyro_skew,
                                dancer, move], axis=1)
    return extracted_features

In [15]:
extracted_features_training = extract_features(dance_move_training)

In [ ]:
extracted_features_training

In [16]:
extracted_features_testing = extract_features(dance_move_testing)

In [ ]:
extracted_features_testing

In [17]:
X = extracted_features_training.iloc[:, :-2]
X_test = extracted_features_testing.iloc[:, :-2]

In [18]:
y = extracted_features_training.iloc[:, -1]
y_test = extracted_features_testing.iloc[:, -1]

In [19]:
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)
y_test = encoder.fit_transform(y_test)

In [20]:
encoder.classes_

array(['dab', 'elbowkick', 'gun', 'hair', 'listen', 'pointhigh',
       'sidepump', 'wipetable'], dtype=object)

In [21]:
train_histories = []
cm_hist = []
classification_report_hist = []

In [22]:
def perform_mlp(X_test, y_test, fold, pca):
    start_time = timer()
    k = fold
    perform_pca = pca
    number_of_classes = 8
    kf = KFold(n_splits=k, shuffle=True)
    #kf = StratifiedKFold(n_splits=k, shuffle=True)

    acc_scores = []
    cv_iteration = 1
    cv_pca_iteration = 1
    train_histories.clear()
    cm_hist.clear()
    classification_report_hist.clear()
    
    for train_index, val_index in kf.split(X):
        
        if perform_pca == False:
            print('\nTraining model and cross validate using fold #{}...\n ' .format(cv_iteration))
            cv_iteration += 1
        
        X_train , X_val = X.iloc[train_index,:], X.iloc[val_index,:]
        y_train , y_val = y[train_index], y[val_index]
        
        X_train = np.array(X_train)
        X_val = np.array(X_val)
        X_test = np.array(X_test)
        print('X_train[0]', X_train[0].shape)
        print('X_val', X_val.shape)
        print('y_train', y_train.shape)
        print('y_val', y_val.shape)
        print('X_test', X_test.shape)
        print('y_test', y_test.shape)

        y_val_without_transform = y_val
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)

        print('after scaling X_train[0]', X_train[0].shape)
        
        y_train = to_categorical(y_train, number_of_classes)
        y_val = to_categorical(y_val, number_of_classes)
        y_test_categorical = to_categorical(y_test, number_of_classes)
        
        if perform_pca == True:
            print('\nTraining model with PCA and cross validate using fold #{}...\n ' .format(cv_pca_iteration))
            cv_pca_iteration += 1
            pca = PCA(n_components = 12)
            X_train = pca.fit_transform(X_train)
            X_val = pca.transform(X_val)
            X_test = pca.transform(X_test)
#             pca.explained_variance_ratio_

        def mlp_model():
            model = Sequential()
#             model.add(Flatten(input_shape=X_train[0].shape))
            model.add(Dense(units=16, kernel_initializer='uniform', activation='relu', input_shape=X_train[0].shape))
#             model.add(Dense(units=64, kernel_initializer='uniform', activation='relu'))
#             model.add(Dropout(0.1))
#             model.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
            model.add(Dropout(0.1))
            model.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
#             model.add(Flatten())
            model.add(Dense(units=number_of_classes, kernel_initializer='uniform', activation='softmax'))
            model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
            return model
        
        mlp = mlp_model()
        print(mlp.summary())        
        
#         checkpoint_filepath="MLP_weights_checkpoint.hdf5"
                
        my_callbacks = [
            EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20),
            ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_delta=0.00001, patience=20, verbose=1),
#             ModelCheckpoint(filepath = checkpoint_filepath, save_weights_only=True, monitor='val_accuracy',
#                             verbose=1, save_best_only=True, mode='max')  
        ] 
        
        print('X_train', X_train.shape)
        print('X_val', X_val.shape)
        print('y_train', y_train.shape)
        print('y_val', y_val.shape)
        print('X_test', X_test.shape)
        print('y_test', y_test.shape)
#         print('scaled X_test', X_test.shape)
        
        history = mlp.fit(X_train, y_train, batch_size=32, epochs=200, validation_data=(X_val, y_val),
                                  callbacks=[my_callbacks], shuffle=True)
        
        mlp_pred = np.argmax(mlp.predict(X_test), axis=-1)
        scores = mlp.evaluate(X_test, y_test_categorical, batch_size=32, verbose=0)
        acc_scores.append(scores[1])
        train_histories.append(history.history)
        
#         mlp_weights = mlp.get_weights()
#         print("MLP Weights:", mlp_weights)
        
#         mlp.save('saved_models/MLP_99.6_accuracy')
        
        print('y_test\n', y_test)
        print('')
        print('mlp_pred\n', mlp_pred)
        
        cm_hist.append(confusion_matrix(y_test, mlp_pred))
        classification_report_hist.append(classification_report(y_test, mlp_pred, target_names=encoder.classes_))
        

    end_time = timer()
    time_taken = end_time - start_time

    return mlp, acc_scores, time_taken

In [23]:
mlp_model, acc_scores, time_taken = perform_mlp(X_test, y_test, fold=5, pca=False)


Training model and cross validate using fold #1...
 
X_train[0] (32,)
X_val (2721, 32)
y_train (10883,)
y_val (2721,)
X_test (2736, 32)
y_test (2736,)
after scaling X_train[0] (32,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 936
Trainable params: 936
Non-trainable params: 0
_________________________________________________________________
None
X_train (10883, 32)
X_val (2721, 32)
y_train (10883, 8)
y_val (2721,

341/341 [==============================] - 0s 981us/step - loss: 0.0291 - accuracy: 0.9917 - val_loss: 0.0073 - val_accuracy: 0.9996
Epoch 50/200
341/341 [==============================] - 0s 922us/step - loss: 0.0321 - accuracy: 0.9903 - val_loss: 0.0070 - val_accuracy: 0.9996
Epoch 51/200
341/341 [==============================] - 0s 874us/step - loss: 0.0343 - accuracy: 0.9902 - val_loss: 0.0067 - val_accuracy: 0.9996
Epoch 52/200
341/341 [==============================] - 0s 898us/step - loss: 0.0346 - accuracy: 0.9874 - val_loss: 0.0065 - val_accuracy: 0.9996
Epoch 53/200
341/341 [==============================] - 0s 857us/step - loss: 0.0273 - accuracy: 0.9918 - val_loss: 0.0062 - val_accuracy: 0.9996
Epoch 54/200
341/341 [==============================] - 0s 750us/step - loss: 0.0307 - accuracy: 0.9915 - val_loss: 0.0060 - val_accuracy: 0.9993
Epoch 55/200
341/341 [==============================] - 0s 852us/step - loss: 0.0320 - accuracy: 0.9900 - val_loss: 0.0058 - val_accuracy

Epoch 105/200
341/341 [==============================] - 0s 853us/step - loss: 0.0170 - accuracy: 0.9943 - val_loss: 0.0023 - val_accuracy: 1.0000

Epoch 00105: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
Epoch 106/200
341/341 [==============================] - 0s 737us/step - loss: 0.0169 - accuracy: 0.9940 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 107/200
341/341 [==============================] - 0s 813us/step - loss: 0.0224 - accuracy: 0.9933 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 108/200
341/341 [==============================] - 0s 861us/step - loss: 0.0184 - accuracy: 0.9947 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 109/200
341/341 [==============================] - 0s 765us/step - loss: 0.0173 - accuracy: 0.9960 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 110/200
341/341 [==============================] - 0s 739us/step - loss: 0.0167 - accuracy: 0.9955 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 111/200
341/341 [==============

Epoch 28/200
341/341 [==============================] - 0s 907us/step - loss: 0.0628 - accuracy: 0.9830 - val_loss: 0.0277 - val_accuracy: 0.9956
Epoch 29/200
341/341 [==============================] - 0s 857us/step - loss: 0.0620 - accuracy: 0.9810 - val_loss: 0.0254 - val_accuracy: 0.9956
Epoch 30/200
341/341 [==============================] - 0s 744us/step - loss: 0.0608 - accuracy: 0.9833 - val_loss: 0.0249 - val_accuracy: 0.9956
Epoch 31/200
341/341 [==============================] - 0s 913us/step - loss: 0.0562 - accuracy: 0.9835 - val_loss: 0.0225 - val_accuracy: 0.9956
Epoch 32/200
341/341 [==============================] - 0s 983us/step - loss: 0.0553 - accuracy: 0.9849 - val_loss: 0.0204 - val_accuracy: 0.9956
Epoch 33/200
341/341 [==============================] - 0s 738us/step - loss: 0.0523 - accuracy: 0.9851 - val_loss: 0.0192 - val_accuracy: 0.9960
Epoch 34/200
341/341 [==============================] - 0s 792us/step - loss: 0.0543 - accuracy: 0.9845 - val_loss: 0.0183 -

341/341 [==============================] - 0s 843us/step - loss: 0.0164 - accuracy: 0.9951 - val_loss: 0.0036 - val_accuracy: 0.9993
Epoch 85/200
341/341 [==============================] - 0s 851us/step - loss: 0.0162 - accuracy: 0.9960 - val_loss: 0.0037 - val_accuracy: 0.9993
Epoch 86/200
341/341 [==============================] - 0s 829us/step - loss: 0.0162 - accuracy: 0.9957 - val_loss: 0.0036 - val_accuracy: 0.9993
Epoch 87/200
341/341 [==============================] - 0s 983us/step - loss: 0.0194 - accuracy: 0.9938 - val_loss: 0.0036 - val_accuracy: 0.9993
Epoch 88/200
341/341 [==============================] - 0s 1ms/step - loss: 0.0130 - accuracy: 0.9972 - val_loss: 0.0035 - val_accuracy: 0.9993
Epoch 89/200
341/341 [==============================] - 0s 851us/step - loss: 0.0149 - accuracy: 0.9953 - val_loss: 0.0035 - val_accuracy: 0.9993
Epoch 90/200
341/341 [==============================] - 0s 954us/step - loss: 0.0137 - accuracy: 0.9960 - val_loss: 0.0033 - val_accuracy: 

...

/Users/yeapcl/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


341/341 [==============================] - 1s 1ms/step - loss: 2.0760 - accuracy: 0.2502 - val_loss: 2.0319 - val_accuracy: 0.4489
Epoch 2/200
341/341 [==============================] - 0s 841us/step - loss: 1.9749 - accuracy: 0.4477 - val_loss: 1.7302 - val_accuracy: 0.4643
Epoch 3/200
341/341 [==============================] - 0s 837us/step - loss: 1.6311 - accuracy: 0.4824 - val_loss: 1.3871 - val_accuracy: 0.5824
Epoch 4/200
341/341 [==============================] - 0s 1ms/step - loss: 1.3165 - accuracy: 0.5950 - val_loss: 1.1601 - val_accuracy: 0.6724
Epoch 5/200
341/341 [==============================] - 0s 793us/step - loss: 1.1078 - accuracy: 0.6924 - val_loss: 0.9759 - val_accuracy: 0.7772
Epoch 6/200
341/341 [==============================] - 0s 928us/step - loss: 0.9366 - accuracy: 0.7638 - val_loss: 0.8064 - val_accuracy: 0.8062
Epoch 7/200
341/341 [==============================] - 0s 869us/step - loss: 0.7915 - accuracy: 0.7852 - val_loss: 0.6550 - val_accuracy: 0.8254
E

341/341 [==============================] - 0s 718us/step - loss: 0.0283 - accuracy: 0.9924 - val_loss: 0.0057 - val_accuracy: 0.9996
Epoch 58/200
341/341 [==============================] - 0s 844us/step - loss: 0.0269 - accuracy: 0.9918 - val_loss: 0.0055 - val_accuracy: 0.9996
Epoch 59/200
341/341 [==============================] - 0s 852us/step - loss: 0.0276 - accuracy: 0.9908 - val_loss: 0.0053 - val_accuracy: 0.9996
Epoch 60/200
341/341 [==============================] - 0s 725us/step - loss: 0.0334 - accuracy: 0.9914 - val_loss: 0.0054 - val_accuracy: 0.9996
Epoch 61/200
341/341 [==============================] - 0s 743us/step - loss: 0.0288 - accuracy: 0.9908 - val_loss: 0.0052 - val_accuracy: 0.9996
Epoch 62/200
341/341 [==============================] - 0s 794us/step - loss: 0.0270 - accuracy: 0.9924 - val_loss: 0.0050 - val_accuracy: 0.9996
Epoch 63/200
341/341 [==============================] - 0s 828us/step - loss: 0.0262 - accuracy: 0.9921 - val_loss: 0.0051 - val_accuracy

341/341 [==============================] - 0s 786us/step - loss: 0.0275 - accuracy: 0.9921 - val_loss: 0.0038 - val_accuracy: 0.9996
Epoch 112/200
341/341 [==============================] - 0s 704us/step - loss: 0.0204 - accuracy: 0.9947 - val_loss: 0.0038 - val_accuracy: 0.9996
Epoch 113/200
341/341 [==============================] - 0s 795us/step - loss: 0.0182 - accuracy: 0.9955 - val_loss: 0.0038 - val_accuracy: 0.9996
Epoch 114/200
341/341 [==============================] - 0s 841us/step - loss: 0.0277 - accuracy: 0.9919 - val_loss: 0.0038 - val_accuracy: 0.9996
Epoch 115/200
341/341 [==============================] - 0s 757us/step - loss: 0.0242 - accuracy: 0.9914 - val_loss: 0.0038 - val_accuracy: 0.9996
Epoch 116/200
341/341 [==============================] - 0s 749us/step - loss: 0.0204 - accuracy: 0.9943 - val_loss: 0.0038 - val_accuracy: 0.9996
Epoch 117/200
341/341 [==============================] - 0s 817us/step - loss: 0.0276 - accuracy: 0.9919 - val_loss: 0.0038 - val_ac

/Users/yeapcl/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
print('MLP - prediction accuracy of each fold:\n {}'.format(acc_scores))
print('')
print('MLP - average accuracy in 5-fold = {} with std. deviation {}' .format(np.mean(acc_scores), np.std(acc_scores)))
print('')
print('Time taken: {:.2f} seconds' .format(time_taken))

MLP - prediction accuracy of each fold:
 [0.9788011908531189, 0.6418128609657288, 0.4389619827270508, 0.37097951769828796, 0.2722953259944916]

MLP - average accuracy in 5-fold = 0.5405701756477356 with std. deviation 0.25034101346249416

Time taken: 205.36 seconds


In [ ]:
from extractionUtils import *
IMPORTED_BY_OTHER_MOD = True
from relayMain import DeviceDataBuffer , MIN_MOVE_TIME_SAMPLES

# Definitions
BASEPATH = os.getcwd()
# SAVEFOLDER = 'raw'
# TESTFOLDER = 'testRaw'
PROCESSEDFOLDER = 'processed'
DANCEMOVENAMES = ("dab","elbowkick","gun","hair","listen","pointhigh","sidepump","wipetable")
IS_RETURN_DATAFRAME = True

### ===========PROCESSING CONFIG=============== ###

#Number of samples to include in dance move after end detected
NUMBER_OF_AFTER_SAMPLES = 0
#Number of samples to include in dance move before start detected
NUMBER_OF_BEFORE_SAMPLES = 20
#Minimum number of samples to be considered a move. Set this too low and you will get garbage dance samples
MINIMUM_MOVE_TIME = 40

#Pad recordings that are too short (ie. recording ends too quickly, not enough neutral samples)
IS_PAD = False
#Number of samples to pad
PAD_NUM =20

#If true, all dance moves will contain MAX_SAMPLES number of samples
USE_MAX_SAMPLES = False
#Number of samples per dance move
MAX_SAMPLES = 100


#Use temporal data augmentation (does not make sense for final evaluation architecture)
USETEMP = True
#Number of temporal Augmentations
TEMPORALDATAAUGNUM = 5

#Use acceleration Data Augmentation
USEACCEL= True
#Number of acceleration Data Augmentations
NUMRANDOMSHIFTSACCEL = 20
#Maximum shift in the z axis for acceleration Data Augmentation
Z_RAND_MAX = 0.2

class tempDataBuffer(DeviceDataBuffer):
    def __init__(self, devName, buffersize):
        super().__init__(devName, internalBufferSize = buffersize)

    def updatePredictor(self):
        return None
    def updateRecorder(self):
        return None
    def sendSample(self):
        return None

def listFiles(savepath):
    filepath = os.path.join(BASEPATH, savepath)
    return os.listdir(filepath)

def extractFileMetadata(fname):
    device, movename, timestamp = fname.split("_")
    return (device, movename, timestamp)

def readRawDataset(fname, savepath):
    filepath = os.path.join(BASEPATH, savepath, fname)

    dataset = {
        'a_xList': [],
        'a_yList': [],
        'a_zList': [],
        'g_xList': [],
        'g_yList': [],
        'g_zList': [],
        'activation_List': []
    }
    with open(filepath, 'r', newline='') as csvfile:
        csvReader = csv.reader(csvfile, delimiter=',')
        count = 0
        for row in csvReader:
            # print(row)
            if count == 0:
                count += 1
                continue
            dataset['a_xList'].append(float(row[0]))
            dataset['a_yList'].append(float(row[1]))
            dataset['a_zList'].append(float(row[2]))
            dataset['g_xList'].append(float(row[3]))
            dataset['g_yList'].append(float(row[4]))
            dataset['g_zList'].append(float(row[5]))
            dataset['activation_List'].append(int(row[6]))
            count += 1
        if IS_PAD:
            for i in range(PAD_NUM):
                dataset['a_xList'].insert(0,dataset['a_xList'][0])
                dataset['a_yList'].insert(0,dataset['a_yList'][0])
                dataset['a_zList'].insert(0,dataset['a_zList'][0])
                dataset['g_xList'].insert(0,dataset['g_xList'][0])
                dataset['g_yList'].insert(0,dataset['g_yList'][0])
                dataset['g_zList'].insert(0,dataset['g_zList'][0])
                dataset['activation_List'].insert(0,dataset['activation_List'][0])

                dataset['a_xList'].append(dataset['a_xList'][-1])
                dataset['a_yList'].append(dataset['a_yList'][-1])
                dataset['a_zList'].append(dataset['a_zList'][-1])
                dataset['g_xList'].append(dataset['g_xList'][-1])
                dataset['g_yList'].append(dataset['g_yList'][-1])
                dataset['g_zList'].append(dataset['g_zList'][-1])
                dataset['activation_List'].append(dataset['activation_List'][-1])


        device, movename, timestamp = extractFileMetadata(fname)
        print("Recording from {} with move {} at {} opened with {} samples".format(device, movename,timestamp ,count - 1))

        r = rawDataset(device, movename, timestamp, dataset)

        recalulated = recaluculateDatasetActivations(device + "recal", movename, timestamp, r)

        # r.plot()
        # recalulated.plot()
        # plt.show()
        # plt.clf()
        return recalulated

def recaluculateDatasetActivations(devName, movename, timestamp, d: rawDataset):
    devData = d.dataset

    mybuff = tempDataBuffer(devName, len(devData['a_xList']))
    
    for i in range(len(devData['a_xList'])):
        aX = devData['a_xList'][i]
        aY = devData['a_yList'][i]
        aZ = devData['a_zList'][i]
        gX = devData['g_xList'][i]
        gY = devData['g_yList'][i]
        gZ = devData['g_zList'][i]
        mybuff.updateInternalState(aX,aY,aZ,gX,gY,gZ)
    dataset = {
        'a_xList': mybuff.aX,
        'a_yList': mybuff.aY,
        'a_zList': mybuff.aZ,
        'g_xList': mybuff.gX,
        'g_yList': mybuff.gY,
        'g_zList': mybuff.gZ,
        'activation_List': mybuff.actList
    }
    return rawDataset(devName, movename, timestamp, dataset)


    


def isolateSequences(rawdata, useAccelBaseValueAugmentation =True , useTemporalAugmentation =True):
    moveIdxs = []
    device = rawdata.device
    movename = rawdata.movename
    timestamp = rawdata.timestamp
    d = rawdata.dataset

    numberOfSamples = len(d['a_xList'])
    isInMove=False
    startIdx = None
    endIdx = None
    for idx in range(numberOfSamples):
        currentActivation = d['activation_List'][idx]
        if (currentActivation == 2) and (isInMove == False):
            isInMove = True
            startIdx = idx
            cooldown = MINIMUM_MOVE_TIME
        elif (isInMove == True) and ((not currentActivation == 2) or idx == numberOfSamples-1 ):
            isInMove = False
            endIdx = idx
            moveIdxs.append( (startIdx,endIdx ) )

    movesData = []
    for start,end in moveIdxs:
        
        if (end - start) < MINIMUM_MOVE_TIME:
            print("???")
            continue

        if USE_MAX_SAMPLES:
            if useTemporalAugmentation:
                for i in range(-TEMPORALDATAAUGNUM*2,(TEMPORALDATAAUGNUM+1)*10,2 ):
                    localStart = start - NUMBER_OF_BEFORE_SAMPLES
                    requiresPad = 0
                    if localStart<0:
                        requiresPad = abs(localStart)
                        localStart = 0
                    localEnd = localStart + MAX_SAMPLES
                    a_xList = d['a_xList'][localStart+i:localEnd+i]
                    a_yList = d['a_yList'][localStart+i:localEnd+i]
                    a_zList = d['a_zList'][localStart+i:localEnd+i]
                    g_xList = d['g_xList'][localStart+i:localEnd+i]
                    g_yList = d['g_yList'][localStart+i:localEnd+i]
                    g_zList = d['g_zList'][localStart+i:localEnd+i]
                    activation_List = d['activation_List'][localStart+i:localEnd+i]

                    if len(a_xList)==0:
                        print("DATAERROR")
                        print(len(a_xList),len(a_yList),len(a_zList),len(g_xList),len(g_yList),len(g_zList) )
                        print(len(d['a_xList']),len( d['a_yList']),len(d['a_zList']),len(d['g_xList']),len(d['g_yList']),len(d['g_zList']) )
                        print(localStart+i,localEnd+i)
                        assert False
                    dm = dancemove(device, movename, timestamp,a_xList,a_yList,a_zList,g_xList,g_yList,g_zList,activation_List)
                    movesData.append(dm)
                if useAccelBaseValueAugmentation:
                    for i in range(NUMRANDOMSHIFTSACCEL):
                        localStart = start - NUMBER_OF_BEFORE_SAMPLES
                        localEnd = localStart + MAX_SAMPLES

                        z_rand = random.uniform(0, Z_RAND_MAX)
                        y_rand = random.uniform(0, z_rand)
                        x_rand = z_rand - y_rand

                        a_xList = list( map( lambda x: x + x_rand, a_xList) )
                        a_yList = list( map( lambda x: x + y_rand, a_yList) )
                        a_zList = list( map( lambda x: x - z_rand, a_zList) )

                        # g_xList = d['g_xList'][localStart:localEnd]
                        # g_yList = d['g_yList'][localStart:localEnd]
                        # g_zList = d['g_zList'][localStart:localEnd]
                        # activation_List = d['activation_List'][localStart:localEnd]
                        dm = dancemove(device, movename, timestamp,a_xList,a_yList,a_zList,g_xList,g_yList,g_zList,activation_List)
                        movesData.append(dm)

            if not useAccelBaseValueAugmentation and not useTemporalAugmentation:
                    localStart = start - NUMBER_OF_BEFORE_SAMPLES
                    localEnd = localStart + MAX_SAMPLES
                    a_xList = d['a_xList'][localStart:localEnd]
                    a_yList = d['a_yList'][localStart:localEnd]
                    a_zList = d['a_zList'][localStart:localEnd]
                    g_xList = d['g_xList'][localStart:localEnd]
                    g_yList = d['g_yList'][localStart:localEnd]
                    g_zList = d['g_zList'][localStart:localEnd]
                    activation_List = d['activation_List'][localStart:localEnd]
                    dm = dancemove(device, movename, timestamp,a_xList,a_yList,a_zList,g_xList,g_yList,g_zList,activation_List)
                    movesData.append(dm)

        else:
            
            a_xList = d['a_xList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            a_yList = d['a_yList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            a_zList = d['a_zList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            g_xList = d['g_xList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            g_yList = d['g_yList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            g_zList = d['g_zList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            activation_List = d['activation_List'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]

            dm = dancemove(device, movename, timestamp,a_xList,a_yList,a_zList,g_xList,g_yList,g_zList,activation_List)
            movesData.append(dm)
    return movesData

def processData(dancer, testset =False):
    if not testset:
        SAVEFOLDER = 'dataset/raw/train/' + dancer
        raws = list(map(lambda x:readRawDataset(x, SAVEFOLDER), listFiles(SAVEFOLDER)))
#         raws = list(map(lambda x:readRawDataset(x), listFiles()))
    else:
        TESTFOLDER = 'dataset/raw/test/' + dancer
        raws = list(map(lambda x:readRawDataset(x ,savepath= TESTFOLDER), listFiles(savepath= TESTFOLDER)))
#         raws = list(map(lambda x:readRawDataset(x ,savepath= TESTFOLDER), listFiles(savepath= TESTFOLDER)))

    combinedList = []
    numberOfMoves = 0
    for item in raws:
        # item.plot()
        moves = isolateSequences(item, useTemporalAugmentation = USETEMP, useAccelBaseValueAugmentation= USEACCEL )
        numberOfMoves += len(moves)
        combinedList.extend(moves)
    print("Done")
    print("Extracted {} moves from {} raw data Sequences.".format(numberOfMoves, len(raws)))

    numberOfEachMoves = dict.fromkeys(DANCEMOVENAMES ,0)
    numberOfEachMoves["defaultMove"] = 0
    for item in combinedList:
        numberOfEachMoves[item.movename] += 1
        
    for k,v in numberOfEachMoves.items():
        print("{}  {}".format(v, k))

    # for item in combinedList:
    #     item.plot()

    # for idx,item in enumerate(combinedList):
    #     item.writeThisFile(idx)   
  
    if IS_RETURN_DATAFRAME:
        frame = pd.DataFrame()
        c = list(map(lambda x : x.toDict(),combinedList ))
        for df in c:
            frame = frame.append(df, ignore_index=True)
        return frame

    return combinedList

# if __name__ == "__main__":
#     danceMoveDataset = processData(testset=False)

#     indexedMoves = {}
#     for idx,dance in enumerate(DANCEMOVENAMES):
#         indexedMoves[dance] = idx +1

#     x = []
#     y = []
#     for i in danceMoveDataset:
#         i.extractAllFeatures()
#         # i.plotFeatures(show = False)
#         flatArray = np.array(i.flatFeatures)
#         print(flatArray.shape)
#         x.extend(flatArray)
#         y.extend([indexedMoves[i.movename]]*flatArray.shape[0])
    
#     arrX = np.array(x)
#     print(arrX.shape)

#     print(np.amax(arrX, axis = 0) , np.amax(arrX, axis = 0).shape )
#     print(np.amin(arrX, axis = 0),np.amin(arrX, axis = 0).shape )

#     # pca = decomposition.PCA(n_components=17)
#     # pca.fit(x)
#     # x = pca.transform(x)
    
#     # fig = plt.figure(1, figsize=(4, 3))
#     # ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

#     # ax.scatter(x[:, 0], x[:, 1], x[:, 2], c=y, cmap=plt.cm.nipy_spectral,
#     #         edgecolor='k')
#     # plt.show()
#     # plt.clf()

In [4]:
#18 April 11:30am

import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox, Button, RadioButtons
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec
from sklearn import preprocessing
from sklearn import decomposition
from extractionUtils import *
IMPORTED_BY_OTHER_MOD = True
from relayMain import *

# Definitions
BASEPATH = os.getcwd()
PROCESSEDFOLDER = 'processed'
DANCEMOVENAMES = ("dab","elbowkick","gun","hair","listen","pointhigh","sidepump","wipetable")
IS_RETURN_DATAFRAME = True

### ===========PROCESSING CONFIG=============== ###

#Number of samples to include in dance move after end detected
NUMBER_OF_AFTER_SAMPLES = 0
#Number of samples to include in dance move before start detected
NUMBER_OF_BEFORE_SAMPLES = 5
# NUMBER_OF_BEFORE_SAMPLES = 5
#Minimum number of samples to be considered a move. Set this too low and you will get garbage dance samples
MINIMUM_MOVE_TIME = 25
#Pad recordings that are too short (ie. recording ends too quickly, not enough neutral samples)
IS_PAD = True
#Number of samples to pad
PAD_NUM =20
#If true, all dance moves will contain MAX_SAMPLES number of samples
USE_MAX_SAMPLES = True
#Number of samples per dance move
MAX_SAMPLES = 100
#Use temporal data augmentation (does not make sense for final evaluation architecture)
USETEMP = True
#Number of temporal Augmentations
TEMPORALDATAAUGNUM = 5
#Use acceleration Data Augmentation
USEACCEL= True
#Number of acceleration Data Augmentations
NUMRANDOMSHIFTSACCEL = 5
#Maximum shift in the z axis for acceleration Data Augmentation
Z_RAND_MAX = 0.05

class tempDataBuffer(DeviceDataBuffer):
    def __init__(self, devName, buffersize):
        super().__init__(devName, internalBufferSize = buffersize)
    def updatePredictor(self):
        return None
    def updateRecorder(self):
        return None
    def sendSample(self):
        return None
def listFiles(savepath):
    filepath = os.path.join(BASEPATH, savepath)
    return os.listdir(filepath)
def extractFileMetadata(fname):
    device, movename, timestamp = fname.split("_")
    return (device, movename, timestamp)
def readRawDataset(fname, savepath):
    filepath = os.path.join(BASEPATH, savepath, fname)
    dataset = {
        'a_xList': [],
        'a_yList': [],
        'a_zList': [],
        'g_xList': [],
        'g_yList': [],
        'g_zList': [],
        'activation_List': []
    }
    with open(filepath, 'r', newline='') as csvfile:
        csvReader = csv.reader(csvfile, delimiter=',')
        count = 0
        for row in csvReader:
            # print(row)
            if count == 0:
                count += 1
                continue
            dataset['a_xList'].append(float(row[0]))
            dataset['a_yList'].append(float(row[1]))
            dataset['a_zList'].append(float(row[2]))
            dataset['g_xList'].append(float(row[3]))
            dataset['g_yList'].append(float(row[4]))
            dataset['g_zList'].append(float(row[5]))
            dataset['activation_List'].append(int(row[6]))
            count += 1
        if IS_PAD:
            for i in range(PAD_NUM):
                dataset['a_xList'].insert(0,dataset['a_xList'][0])
                dataset['a_yList'].insert(0,dataset['a_yList'][0])
                dataset['a_zList'].insert(0,dataset['a_zList'][0])
                dataset['g_xList'].insert(0,dataset['g_xList'][0])
                dataset['g_yList'].insert(0,dataset['g_yList'][0])
                dataset['g_zList'].insert(0,dataset['g_zList'][0])
                dataset['activation_List'].insert(0,dataset['activation_List'][0])
                dataset['a_xList'].append(dataset['a_xList'][-1])
                dataset['a_yList'].append(dataset['a_yList'][-1])
                dataset['a_zList'].append(dataset['a_zList'][-1])
                dataset['g_xList'].append(dataset['g_xList'][-1])
                dataset['g_yList'].append(dataset['g_yList'][-1])
                dataset['g_zList'].append(dataset['g_zList'][-1])
                dataset['activation_List'].append(dataset['activation_List'][-1])
        device, movename, timestamp = extractFileMetadata(fname)
        print("Recording from {} with move {} at {} opened with {} samples".format(device, movename,timestamp ,count - 1))
        r = rawDataset(device, movename, timestamp, dataset)
        recalulated = recaluculateDatasetActivations(device + "recal", movename, timestamp, r)
        # r.plot()
        # recalulated.plot()
        # plt.show()
        # plt.clf()
        return recalulated

def recaluculateDatasetActivations(devName, movename, timestamp, d: rawDataset):
    devData = d.dataset
    mybuff = tempDataBuffer(devName, len(devData['a_xList']))
    
    for i in range(len(devData['a_xList'])):
        aX = devData['a_xList'][i]
        aY = devData['a_yList'][i]
        aZ = devData['a_zList'][i]
        gX = devData['g_xList'][i]
        gY = devData['g_yList'][i]
        gZ = devData['g_zList'][i]
        mybuff.updateInternalState(aX,aY,aZ,gX,gY,gZ)
    dataset = {
        'a_xList': mybuff.aX,
        'a_yList': mybuff.aY,
        'a_zList': mybuff.aZ,
        'g_xList': mybuff.gX,
        'g_yList': mybuff.gY,
        'g_zList': mybuff.gZ,
        'activation_List': mybuff.actList
    }
    return rawDataset(devName, movename, timestamp, dataset)
    
def isolateSequences(rawdata, useAccelBaseValueAugmentation =True , useTemporalAugmentation =True):
    moveIdxs = []
    device = rawdata.device
    movename = rawdata.movename
    timestamp = rawdata.timestamp
    d = rawdata.dataset
    numberOfSamples = len(d['a_xList'])
    isInMove=False
    startIdx = None
    endIdx = None
    for idx in range(numberOfSamples):
        currentActivation = d['activation_List'][idx]
        if (currentActivation == 2) and (isInMove == False):
            isInMove = True
            startIdx = idx
            cooldown = MINIMUM_MOVE_TIME
        elif (isInMove == True) and ((not currentActivation == 2) or idx == numberOfSamples-1 ):
            isInMove = False
            endIdx = idx
            moveIdxs.append( (startIdx,endIdx ) )
    movesData = []
    for start,end in moveIdxs:
        
        if (end - start) < MINIMUM_MOVE_TIME:
            print("???")
            continue
        if USE_MAX_SAMPLES:
            if useTemporalAugmentation:
                for i in range(-TEMPORALDATAAUGNUM,(TEMPORALDATAAUGNUM+1)*10,2 ):
                    print(len(d['a_xList']))
                    localStart = start - NUMBER_OF_BEFORE_SAMPLES
                    requiresPad = 0
                    if localStart<0:
                        requiresPad = abs(localStart)
                        localStart = 0
                    localEnd = localStart + MAX_SAMPLES
                    a_xList = d['a_xList'][localStart+i:localEnd+i]
                    a_yList = d['a_yList'][localStart+i:localEnd+i]
                    a_zList = d['a_zList'][localStart+i:localEnd+i]
                    g_xList = d['g_xList'][localStart+i:localEnd+i]
                    g_yList = d['g_yList'][localStart+i:localEnd+i]
                    g_zList = d['g_zList'][localStart+i:localEnd+i]
                    activation_List = d['activation_List'][localStart+i:localEnd+i]
                    if len(a_xList)==0 or len(a_xList) != MAX_SAMPLES:
                        print("DATAERROR")
                        print(len(a_xList),len(a_yList),len(a_zList),len(g_xList),len(g_yList),len(g_zList) )
                        print(len(d['a_xList']),len( d['a_yList']),len(d['a_zList']),len(d['g_xList']),len(d['g_yList']),len(d['g_zList']) )
                        print(localStart+i,localEnd+i)
                        assert False
                    dm = dancemove(device, movename, timestamp,a_xList,a_yList,a_zList,g_xList,g_yList,g_zList,activation_List)
                    movesData.append(dm)
                if useAccelBaseValueAugmentation:
                    for i in range(NUMRANDOMSHIFTSACCEL):
                        localStart = start - NUMBER_OF_BEFORE_SAMPLES
                        localEnd = localStart + MAX_SAMPLES
                        z_rand = random.uniform(0, Z_RAND_MAX)
                        y_rand = random.uniform(0, z_rand)
                        x_rand = z_rand - y_rand
                        a_xList = list( map( lambda x: x + x_rand, a_xList) )
                        a_yList = list( map( lambda x: x + y_rand, a_yList) )
                        a_zList = list( map( lambda x: x - z_rand, a_zList) )
                        # g_xList = d['g_xList'][localStart:localEnd]
                        # g_yList = d['g_yList'][localStart:localEnd]
                        # g_zList = d['g_zList'][localStart:localEnd]
                        # activation_List = d['activation_List'][localStart:localEnd]
                        dm = dancemove(device, movename, timestamp,a_xList,a_yList,a_zList,g_xList,g_yList,g_zList,activation_List)
                        movesData.append(dm)
            if not useAccelBaseValueAugmentation and not useTemporalAugmentation:
                    localStart = start - NUMBER_OF_BEFORE_SAMPLES
                    localEnd = localStart + MAX_SAMPLES
                    a_xList = d['a_xList'][localStart:localEnd]
                    a_yList = d['a_yList'][localStart:localEnd]
                    a_zList = d['a_zList'][localStart:localEnd]
                    g_xList = d['g_xList'][localStart:localEnd]
                    g_yList = d['g_yList'][localStart:localEnd]
                    g_zList = d['g_zList'][localStart:localEnd]
                    activation_List = d['activation_List'][localStart:localEnd]
                    dm = dancemove(device, movename, timestamp,a_xList,a_yList,a_zList,g_xList,g_yList,g_zList,activation_List)
                    movesData.append(dm)
        else:
            
            a_xList = d['a_xList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            a_yList = d['a_yList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            a_zList = d['a_zList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            g_xList = d['g_xList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            g_yList = d['g_yList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            g_zList = d['g_zList'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            activation_List = d['activation_List'][start - NUMBER_OF_BEFORE_SAMPLES: end + NUMBER_OF_AFTER_SAMPLES ]
            dm = dancemove(device, movename, timestamp,a_xList,a_yList,a_zList,g_xList,g_yList,g_zList,activation_List)
            movesData.append(dm)
    return movesData
def processData(dancer, testset =False):
    if not testset:
        SAVEFOLDER = 'dataset/raw/train/' + dancer
        raws = list(map(lambda x:readRawDataset(x, SAVEFOLDER), listFiles(SAVEFOLDER)))
#         raws = list(map(lambda x:readRawDataset(x), listFiles()))
    else:
        TESTFOLDER = 'dataset/raw/test/' + dancer
        raws = list(map(lambda x:readRawDataset(x ,savepath= TESTFOLDER), listFiles(savepath= TESTFOLDER)))
#         raws = list(map(lambda x:readRawDataset(x ,savepath= TESTFOLDER), listFiles(savepath= TESTFOLDER)))
    combinedList = []
    numberOfMoves = 0
    for item in raws:
        # item.plot()
        moves = isolateSequences(item, useTemporalAugmentation = USETEMP, useAccelBaseValueAugmentation= USEACCEL )
        if len(moves) == 0:
            # item.plot()
            # plt.show()
            # plt.clf()
            pass
        numberOfMoves += len(moves)
        combinedList.extend(moves)
    print("Done")
    print("Extracted {} moves from {} raw data Sequences.".format(numberOfMoves, len(raws)))
    numberOfEachMoves = dict.fromkeys(DANCEMOVENAMES ,0)
    numberOfEachMoves["defaultMove"] = 0
    for item in combinedList:
        numberOfEachMoves[item.movename] += 1
        
    for k,v in numberOfEachMoves.items():
        print("{}  {}".format(v, k))
    # for item in combinedList:
    #     item.plot()
    # for idx,item in enumerate(combinedList):
    #     item.writeThisFile(idx)   
  
    if IS_RETURN_DATAFRAME:
        frame = pd.DataFrame()
        c = list(map(lambda x : x.toDict(),combinedList ))
        for df in c:
            frame = frame.append(df, ignore_index=True)
        return frame
    return combinedList

RUNNING LOCALLY
